# Final Project: The battle of neighborhoods

### *Install packages*

In [3]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.0-py36_0         conda-forge
    ca-certificates: 2019.5.15-0       --> 2019.6.16-hecc5488_0 conda-forge
    certifi:         2019.6.16-py36_1  --> 2019.6.16-py36_1     conda-forg

### *Import packages*

In [4]:
import pandas as pd 
import numpy as np 
import random 

from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 
    

from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import requests

import folium 

### *Setup folium*

In [5]:
CLIENT_ID = 'TS5OD3HSFDGPLEMNNZUEVKURUU02DVU15KPRZPWCYZVIC1BH' # your Foursquare ID
CLIENT_SECRET = 'IIIJR1510FOMLAE2KB0BRD32YIFH4SHO4UIN5K11ISDDTAQB' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TS5OD3HSFDGPLEMNNZUEVKURUU02DVU15KPRZPWCYZVIC1BH
CLIENT_SECRET:IIIJR1510FOMLAE2KB0BRD32YIFH4SHO4UIN5K11ISDDTAQB


## Introduction

In this report, we are going the answer the question "Which areas in London where a restaurant can be established". In order to answer the question, we will use data science methodology and clustering technique.

### *Scraping data from web*

In [7]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_London_boroughs")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
boroughs = pd.read_html(str(table))[0]

### *cleaning the data by replace function, dropping unused columns and renaming column header*

In [9]:
boroughs['Borough'].replace({'Barking and Dagenham [note 1]':'Barking and Dagenham', 'Greenwich [note 2]':'Greenwich', 'Hammersmith and Fulham [note 4]':'Hammersmith and Fulham'}, inplace=True)
boroughs.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham,NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [10]:
boroughs = boroughs[['Borough','Local authority']]
boroughs.head()

,Borough,Local authority
0,Barking and Dagenham,Barking and Dagenham London Borough Council
1,Barnet,Barnet London Borough Council
2,Bexley,Bexley London Borough Council
3,Brent,Brent London Borough Council
4,Bromley,Bromley London Borough Council


In [11]:
postcode = {'Local authority':['Barking and Dagenham Borough Council', 'Barnet London Borough Council','Bexley London Borough Council','Brent London Borough Council','Bromley London Borough Council', 'Camden London Borough Council','Croydon London Borough Council','Ealing London Borough Council','Enfield London Borough Council','Greenwich London Borough Council','Hackney London Borough Council','Hammersmith and Fulham London Borough Council','Haringey London Borough Council','Harrow London Borough Council', 'Havering London Borough Council','Hillingdon London Borough Council','Hounslow London Borough Council','Islington London Borough Council', 'Kensington and Chelsea London Borough Council','Kingston upon Thames London Borough Council','Lambeth London Borough Council','Lewisham London Borough Council','Merton London Borough Council','Newham London Borough Council','Redbridge London Borough Council','Richmond upon Thames London Borough Council','Southwark London Borough Council','Sutton London Borough Council','Tower Hamlets London Borough Council','Waltham Forest London Borough Council','Wandsworth London Borough Council','Westminster City Council'],
            'Postcode':['RM','N11', 'NaN','NW4','BR','NW1','CR','W5','EN','SE10','E8','W6','NaN','HA','NaN','NaN','NaN','N1','SW3','KT','SE11','SE13','SW19','NaN','NaN','NaN','SE1','SM','NaN','E17','SW18','SW1']}

In [14]:
Postcode=pd.DataFrame(postcode)
Postcode.head()

,Local authority,Postcode
0,Barking and Dagenham Borough Council,RM
1,Barnet London Borough Council,N11
2,Bexley London Borough Council,NaN
3,Brent London Borough Council,NW4
4,Bromley London Borough Council,BR


### Merge postcode and the borough into dataframe

In [15]:
combined = pd.merge(Postcode,boroughs, on=['Local authority'], how='inner')
combined.head()

,Local authority,Postcode,Borough
0,Barnet London Borough Council,N11,Barnet
1,Bexley London Borough Council,NaN,Bexley
2,Brent London Borough Council,NW4,Brent
3,Bromley London Borough Council,BR,Bromley
4,Camden London Borough Council,NW1,Camden


### Drop rows with no postcode

In [17]:
combined = combined[combined.Postcode != 'NaN'].reset_index()
combined = combined[['Local authority','Postcode','Borough']]
combined.head()

,Local authority,Postcode,Borough
0,Barnet London Borough Council,N11,Barnet
1,Brent London Borough Council,NW4,Brent
2,Bromley London Borough Council,BR,Bromley
3,Camden London Borough Council,NW1,Camden
4,Croydon London Borough Council,CR,Croydon


### Import Longitude Latitude data and merge it to existing dataframe

In [18]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude,Easting,Northing,Grid Reference,Town/Area,Region,Postcodes,Active postcodes,Population,Households,Nearby districts
0,AB1,57.1269,-2.13644,391839.0,804005.0,NJ918040,Aberdeen,Aberdeen,2655.0,0.0,NaN,NaN,"AB10, AB15, AB9, AB25, AB2, AB12, AB16, AB11, ..."
1,AB2,57.1713,-2.14152,391541.0,808948.0,NJ915089,Aberdeen,Aberdeen,3070.0,0.0,NaN,NaN,"AB16, AB22, AB24, AB25, AB1, AB9, AB15, AB10, ..."
2,AB3,57.0876,-2.59624,363963.0,799780.0,NO639997,Aberdeen,Aberdeen,2168.0,0.0,NaN,NaN,"AB31, AB34, AB52, AB33, AB14, AB32, AB30, AB13..."
3,AB4,57.5343,-2.12713,392487.0,849358.0,NJ924493,"Fraserburgh, Peterhead, Macduff",Aberdeen,2956.0,0.0,NaN,NaN,"AB41, AB43, AB42, AB53, AB22, AB23, AB2, AB24,..."
4,AB5,57.4652,-2.64764,361248.0,841843.0,NJ612418,"Buckie, Huntly, Inverurie",Aberdeen,3002.0,0.0,NaN,NaN,"AB54, AB52, AB45, AB53, AB33, AB44, AB55, AB51..."


In [26]:
df = pd.merge(combined, df_data_1, on='Postcode', how ='inner')

In [28]:
df = df [['Borough','Local authority','Postcode','Latitude','Longitude']]
df.head()

,Borough,Local authority,Postcode,Latitude,Longitude
0,Barnet,Barnet London Borough Council,N11,51.6142,-0.138362
1,Brent,Brent London Borough Council,NW4,51.5874,-0.223797
2,Camden,Camden London Borough Council,NW1,51.5323,-0.142796
3,Ealing,Ealing London Borough Council,W5,51.5122,-0.304002
4,Greenwich,Greenwich London Borough Council,SE10,51.4840,0.000656


In [29]:
address = 'London'
my_agent ='vn_sg_coursera_capstone'
geolocator = Nominatim(user_agent=my_agent)
location = geolocator.geocode(address)
london_lat = location.latitude
london_longi = location.longitude

# create map of London using latitude and longitude values
map_london = folium.Map(location=[london_lat, london_longi], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes): 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
London_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [33]:
print(London_venues.shape)
London_venues.head()

(1237, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barnet,51.6142,-0.138362,Boyden's Kitchen,51.615233,-0.146136,Café
1,Barnet,51.6142,-0.138362,Pets at Home,51.611235,-0.142142,Pet Store
2,Barnet,51.6142,-0.138362,Sports Direct,51.611507,-0.144498,Sporting Goods Shop
3,Barnet,51.6142,-0.138362,Costa Coffee,51.610912,-0.143461,Coffee Shop
4,Barnet,51.6142,-0.138362,Arnos Park,51.618336,-0.130720,Park


In [249]:
print('There are {} uniques venue categories.'.format(len(London_venues['Venue Category'].unique())))

There are 222 uniques venue categories.


London_venues.groupby('Neighborhood').count()

### Analyze the boroughs

In [40]:
# one hot encoding
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
London_onehot['Neighborhood'] = London_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]
print(London_onehot.shape)
London_onehot.head()

(1237, 226)


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Veterinarian,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Barnet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barnet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barnet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barnet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barnet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
London_grouped = London_onehot.groupby('Neighborhood').mean().reset_index()
London_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Veterinarian,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Barnet,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.045455,0.00,0.00
1,Brent,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.000000,0.00,0.00
2,Camden,0.0,0.01,0.0,0.0,0.01,0.000000,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.010000,0.01,0.07
3,Ealing,0.0,0.00,0.0,0.0,0.00,0.010000,0.0,0.0,0.01,...,0.0,0.01,0.02,0.0,0.02,0.000000,0.0,0.000000,0.00,0.00
4,Greenwich,0.0,0.00,0.0,0.0,0.00,0.021505,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.010753,0.0,0.000000,0.00,0.00


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
London_venues_sorted = pd.DataFrame(columns=columns)
London_venues_sorted['Neighborhood'] = London_grouped['Neighborhood']

for ind in np.arange(London_grouped.shape[0]):
    London_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)
London_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Grocery Store,Park,Convenience Store,Residential Building (Apartment / Condo),Bus Stop,Hardware Store,Gym / Fitness Center,Metro Station,Furniture / Home Store,Fast Food Restaurant
1,Brent,Grocery Store,Park,Coffee Shop,Hotel,Pizza Place,Gym / Fitness Center,Bagel Shop,Middle Eastern Restaurant,Student Center,Sushi Restaurant
2,Camden,Zoo Exhibit,Coffee Shop,Beer Bar,Burger Joint,Music Venue,Theater,Pub,Greek Restaurant,Garden,Café
3,Ealing,Coffee Shop,Pub,Hotel,Park,Italian Restaurant,Bakery,Gym / Fitness Center,Café,Pizza Place,Thai Restaurant
4,Greenwich,Pub,Grocery Store,Café,Garden,Park,Coffee Shop,Burger Joint,Bar,Market,Bakery
5,Hackney,Café,Pub,Bakery,Coffee Shop,Brewery,Restaurant,Pizza Place,Cocktail Bar,Bookstore,Park
6,Hammersmith and Fulham,Pub,Coffee Shop,Indian Restaurant,Café,Italian Restaurant,Park,Japanese Restaurant,Thai Restaurant,Gastropub,French Restaurant
7,Islington,Pub,French Restaurant,Mediterranean Restaurant,Gastropub,Bakery,Boutique,Park,Ice Cream Shop,Music Venue,Burger Joint
8,Kensington and Chelsea,Italian Restaurant,Bakery,French Restaurant,Japanese Restaurant,English Restaurant,Café,Pizza Place,Plaza,Pub,Coffee Shop
9,Lambeth,Pub,Café,Park,Hotel,Gastropub,Coffee Shop,Italian Restaurant,Indian Restaurant,Gay Bar,Art Gallery


# Discussion

The table above shows that the most common venue in the neighborhood is Pub which is a place where people gather in a community. We can see that only one Restaurant appeared in our top venue. The table also shows that Hammersmith and Fulham is the place where the restaurant is a common venue. We are going to choose a neighborhood where there is an opportunity in setting up a restaurant.

# Conclusion

After conditioning our raw data and analyzing foursquare data, we have arrived in our conclusion in where to open up a restaurant.

Our study concludes that Barnet is the ideal neighborhood in opening a restaurant as it has a greater opportunity for us. Currently the restaurant appeared in the 10th place most common venue so it is a great time for us to start a restaurant business.
